In [1]:
library(Seurat)
library(Signac)
library(SeuratWrappers)
library(RENIN)
library(tidyverse)
library(chromVARmotifs)
library(BSgenome.Hsapiens.UCSC.hg19)

Attaching SeuratObject

Attaching sp


Attaching package: ‘Signac’


The following object is masked from ‘package:Seurat’:

    FoldChange


── Attaching packages ─────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.1     ✔ purrr   0.3.4
✔ tibble  3.1.8     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1
── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ purrr::reduce() masks Signac::reduce()


Warning message:
“no function found corresponding to methods exports from ‘BSgenome’ for: ‘releaseName’”
Loading required package: BSgenome

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, cluste

## define the two ident to compare in FindMarkers function

In [2]:
FindMarkers_ident_1 = "Medulla"
FindMarkers_ident_2 = "Cortex"

celltype_subset = c("TAL1", "TAL2", "TAL3")

In [3]:
de_gene_file = "../../processed_data/RENIN/de_genes/446267_cells_RNA/TAL_rank_genes_groups_m_vs_c_20230425.RData"

In [4]:
processed_dir = file.path("..", "..", "processed_data", "RENIN", "by_regions", paste0(gsub(pattern = "\\/", "_", paste(celltype_subset, collapse = "_"))))
processed_dir
dir.create(processed_dir, recursive = T, showWarnings = F)

[1] "../../processed_data/RENIN/by_regions/TAL1_TAL2_TAL3"

### load previously calculated image which includes expr_mat, peak_mat, novaseq.sub. This could save time for dealing with new de_gene for different clusters while sharing the same novaseq.sub object

In [5]:
Sys.time()
load("../../processed_data/RENIN/RENIN_324701_cells_preprocess.RData")
Sys.time()

[1] "2023-07-01 13:19:08 CDT"

[1] "2023-07-01 13:23:56 CDT"

In [6]:
novaseq.sub
table(novaseq.sub$celltype5_rna)

An object of class Seurat 
237522 features across 324701 samples within 2 assays 
Active assay: peaks (189184 features, 189184 variable features)
 1 other assay present: SCT
 6 dimensional reductions calculated: pca, harmony_RNA, lsi, harmony_peaks, umap.peaks, WNN.UMAP


      JGA       POD       PEC        PT PT_dediff  PT_VCAM1       tL1       tL2 
     2465      3179      1239     64880     29494      8445     25438      9029 
   tL-TAL      TAL1      TAL2      TAL3       DCT       CNT       PC1       PC2 
    15485     29910     19724      8900     14150     13197     16140      4049 
      ICA       ICB      Uro1      Uro2      ENDO      SMC1      SMC2      SMC3 
    10579      4120      3210       682     12847      4272       156        86 
     Fib1      Fib2      Fib3        Ma        BT 
     3786      7191      3095      3673      5280 

In [7]:
ls()

[1] "celltype_subset"                  "de_gene_file"                    
 [3] "expr_mat"                         "FindMarkers_ident_1"             
 [5] "FindMarkers_ident_2"              "level.novaseq"                   
 [7] "level.novaseq.renal_region_new"   "level.novaseq.rna"               
 [9] "novaseq.sub"                      "palette.novaseq"                 
[11] "palette.novaseq.renal_region_new" "palette.novaseq.rna"             
[13] "peak_mat"                         "processed_dir"

In [8]:
## subset novaseq.sub again to only include the celltypes of interest

In [9]:
novaseq.sub = subset(novaseq.sub, celltype5_rna %in% celltype_subset)
novaseq.sub
table(novaseq.sub$celltype5_rna)

An object of class Seurat 
237522 features across 58534 samples within 2 assays 
Active assay: peaks (189184 features, 189184 variable features)
 1 other assay present: SCT
 6 dimensional reductions calculated: pca, harmony_RNA, lsi, harmony_peaks, umap.peaks, WNN.UMAP


      JGA       POD       PEC        PT PT_dediff  PT_VCAM1       tL1       tL2 
        0         0         0         0         0         0         0         0 
   tL-TAL      TAL1      TAL2      TAL3       DCT       CNT       PC1       PC2 
        0     29910     19724      8900         0         0         0         0 
      ICA       ICB      Uro1      Uro2      ENDO      SMC1      SMC2      SMC3 
        0         0         0         0         0         0         0         0 
     Fib1      Fib2      Fib3        Ma        BT 
        0         0         0         0         0 

In [10]:
ls()

[1] "celltype_subset"                  "de_gene_file"                    
 [3] "expr_mat"                         "FindMarkers_ident_1"             
 [5] "FindMarkers_ident_2"              "level.novaseq"                   
 [7] "level.novaseq.renal_region_new"   "level.novaseq.rna"               
 [9] "novaseq.sub"                      "palette.novaseq"                 
[11] "palette.novaseq.renal_region_new" "palette.novaseq.rna"             
[13] "peak_mat"                         "processed_dir"

### remove the following variables because they will be regenerated for new comparison

In [11]:
rm(list = c("de_genes", "de_genes_BK", "gene_list", "peak_results", "aen_lists", "tf_results", "regulator_tf_names", "centrality_rankings"))

Warning message in rm(list = c("de_genes", "de_genes_BK", "gene_list", "peak_results", :
“object 'de_genes' not found”
Warning message in rm(list = c("de_genes", "de_genes_BK", "gene_list", "peak_results", :
“object 'de_genes_BK' not found”
Warning message in rm(list = c("de_genes", "de_genes_BK", "gene_list", "peak_results", :
“object 'gene_list' not found”
Warning message in rm(list = c("de_genes", "de_genes_BK", "gene_list", "peak_results", :
“object 'peak_results' not found”
Warning message in rm(list = c("de_genes", "de_genes_BK", "gene_list", "peak_results", :
“object 'aen_lists' not found”
Warning message in rm(list = c("de_genes", "de_genes_BK", "gene_list", "peak_results", :
“object 'tf_results' not found”
Warning message in rm(list = c("de_genes", "de_genes_BK", "gene_list", "peak_results", :
“object 'regulator_tf_names' not found”
Warning message in rm(list = c("de_genes", "de_genes_BK", "gene_list", "peak_results", :
“object 'centrality_rankings' not found”


In [12]:
ls()

[1] "celltype_subset"                  "de_gene_file"                    
 [3] "expr_mat"                         "FindMarkers_ident_1"             
 [5] "FindMarkers_ident_2"              "level.novaseq"                   
 [7] "level.novaseq.renal_region_new"   "level.novaseq.rna"               
 [9] "novaseq.sub"                      "palette.novaseq"                 
[11] "palette.novaseq.renal_region_new" "palette.novaseq.rna"             
[13] "peak_mat"                         "processed_dir"

In [13]:
Idents(novaseq.sub) = novaseq.sub$renal_region_new
head(Idents(novaseq.sub))

R1.056,R2.038,R3.008,P1.48,B2 R1.031,R2.072,R3.075,P1.78,B3 
                      Medulla                       Medulla 
R1.006,R2.001,R3.001,P1.11,B3 R1.048,R2.071,R3.010,P1.66,B2 
                       Cortex                        Cortex 
R1.030,R2.014,R3.064,P1.50,B3 R1.078,R2.071,R3.079,P1.50,B3 
                      Papilla                       Papilla 
Levels: Cortex Medulla Papilla Renal Artery Ureter

### subset mpt from novaseq.sub because comparison is to be made bewtween two celltypes

In [14]:
mpt <- subset(novaseq.sub, renal_region_new %in% c(FindMarkers_ident_1, FindMarkers_ident_2))
mpt
table(mpt$celltype5_rna)

An object of class Seurat 
237522 features across 39908 samples within 2 assays 
Active assay: peaks (189184 features, 189184 variable features)
 1 other assay present: SCT
 6 dimensional reductions calculated: pca, harmony_RNA, lsi, harmony_peaks, umap.peaks, WNN.UMAP


      JGA       POD       PEC        PT PT_dediff  PT_VCAM1       tL1       tL2 
        0         0         0         0         0         0         0         0 
   tL-TAL      TAL1      TAL2      TAL3       DCT       CNT       PC1       PC2 
        0     14771     16600      8537         0         0         0         0 
      ICA       ICB      Uro1      Uro2      ENDO      SMC1      SMC2      SMC3 
        0         0         0         0         0         0         0         0 
     Fib1      Fib2      Fib3        Ma        BT 
        0         0         0         0         0 

## load hli_446267_cells de_gene data frame

In [15]:
Sys.time()
options(future.globals.maxSize = 8000 * 1024^2)
load(de_gene_file)
Sys.time()

[1] "2023-07-01 13:24:57 CDT"

[1] "2023-07-01 13:24:57 CDT"

In [16]:
rownames(de_genes) = de_genes$names
de_genes_BK = de_genes
print(head(de_genes))
dim(de_genes)

          names           scores pvals pvals_adj    logfoldchanges
PDE1A     PDE1A 75.1906814575195     0         0  1.43420672416687
MAST4     MAST4 70.0619735717773     0         0  1.35592079162598
ESRRG     ESRRG 69.1707611083984     0         0 0.981366038322449
FOXP2     FOXP2 68.5606842041016     0         0  1.95928347110748
SLC4A7   SLC4A7 68.4411849975586     0         0  2.02226448059082
SLC12A1 SLC12A1 65.9282150268555     0         0  1.06851625442505


[1] 40634     5

In [17]:
### keep DEGs with pvals_adj<0.05
p_val_cutoff = 0.05
peak_assay = "peaks"
de_genes <- de_genes[which(de_genes$pvals_adj < p_val_cutoff), ]
DefaultAssay(novaseq.sub) <- peak_assay
DefaultAssay(mpt) <- peak_assay

dim(de_genes)

[1] 1919    5

In [18]:
Sys.time()
# gene_coords <- Signac:::CollapseToLongestTranscript(Annotation(novaseq.sub))
gene_coords <- Signac:::CollapseToLongestTranscript(Annotation(mpt))
Sys.time()

[1] "2023-07-01 13:24:57 CDT"

[1] "2023-07-01 13:25:00 CDT"

In [19]:
de_genes <- de_genes[which(rownames(de_genes) %in% gene_coords$gene_name), ]
dim(de_genes)
gene_list = rownames(de_genes)

[1] 1504    5

## Next, make two de_gene sublists: gene_list_neg, gene_list_pos.

In [20]:
gene_list_neg = rownames(de_genes)[de_genes$logfoldchanges < 0]
print(length(gene_list_neg))
gene_list_pos = rownames(de_genes)[de_genes$logfoldchanges > 0]
print(length(gene_list_pos))

[1] 369
[1] 1135


In [21]:
# de_genes[order(de_genes$logfoldchanges, decreasing = T),]

## run_peak_aen

In [22]:
Sys.time()

options(future.globals.maxSize = 8000 * 1024^2)

peak_results <- run_peak_aen(novaseq.sub, expr_mat, peak_mat, gene_list, lambda2 = 0.5, max_distance = 5e+05, num_bootstraps = 100)

aen_lists <- make_aen_lists(peak_results)

Sys.time()

[1] "2023-07-01 13:25:01 CDT"

Loading required package: gcdnet


Attaching package: ‘gcdnet’


The following object is masked from ‘package:stats4’:

    coef


The following objects are masked from ‘package:stats’:

    coef, predict


Loading required package: future


Attaching package: ‘future’


The following object is masked from ‘package:rtracklayer’:

    values


The following object is masked from ‘package:GenomicRanges’:

    values


The following object is masked from ‘package:IRanges’:

    values


The following object is masked from ‘package:S4Vectors’:

    values


Loading required package: future.apply



[1] "Omitted (mitochondrial) genes with no genomic peaks: SMN1"  
[2] "Omitted (mitochondrial) genes with no genomic peaks: TTTY10"
[3] "Omitted (mitochondrial) genes with no genomic peaks: MT-CO3"
[4] "Omitted (mitochondrial) genes with no genomic peaks: MT-ND3"
[5] "Omitted (mitochondrial) genes with no genomic peaks: MT-ND6"
[6] "Omitted (mitochondrial) genes with no genomic peaks: MT-CO2"
[1] "AEN completed in 11.3215464353561"


[1] "2023-07-01 13:36:55 CDT"

## prepare_pseudocell_matrix for mpt

In [23]:
expr_mat_2 <- prepare_pseudocell_matrix(mpt, 
                                      assay = "SCT", 
                                      slot = "data", 
                                      cells_per_partition = 10, 
                                      find_neighbors = FALSE,
                                      reduction1 = "harmony_peaks",
                                      reduction2 = "harmony_RNA",
                                      dim_list = list(1:50, 1:50),
                                      k.nn = 5,
                                      seed = 489284)

Loading required package: Matrix


Attaching package: ‘Matrix’


The following object is masked from ‘package:S4Vectors’:

    expand


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack


Loading required package: VISION

Loading required package: SingleCellExperiment

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘matrixStats’


The following object is masked from ‘package:dplyr’:

    count



Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDif

In [24]:
length(gene_list)
sum(gene_list %in% colnames(expr_mat_2))

[1] 1504

[1] 1504

### run_tf_aen

In [25]:
Sys.time()

tf_results <- run_tf_aen(mpt, expr_mat_2, peak_results, gene_list, lambda2 = 0.5)

regulator_tf_names <- unlist(novaseq.sub@assays$peaks@motifs@motif.names)

regulator_tf_names <- regulator_tf_names[which(regulator_tf_names %in% rownames(GetAssayData(novaseq.sub, assay = "SCT")))]

centrality_rankings <- rank_tfs_by_centrality(tf_results, novaseq.sub)

Sys.time()

[1] "2023-07-01 13:45:05 CDT"

[1] "Genes without any candidate TFs--most likely no linked peaks or promoter region peaks: AGAP6" 
[2] "Genes without any candidate TFs--most likely no linked peaks or promoter region peaks: SMN2"  
[3] "Genes without any candidate TFs--most likely no linked peaks or promoter region peaks: ZBTB11"
[4] "Genes without any candidate TFs--most likely no linked peaks or promoter region peaks: NPIPB6"
[5] "Genes without any candidate TFs--most likely no linked peaks or promoter region peaks: BTBD2" 
[6] "Genes without any candidate TFs--most likely no linked peaks or promoter region peaks: RPL10" 
[7] "Genes without any candidate TFs--most likely no linked peaks or promoter region peaks: ZNF701"
[8] "Genes without any candidate TFs--most likely no linked peaks or promoter region peaks: OR7C1" 
[9] "Genes without any candidate TFs--most likely no linked peaks or promoter region peaks: FRMPD3"
[1] "AEN completed in 30.121244764328"


Loading required package: igraph


Attaching package: ‘igraph’


The following objects are masked from ‘package:future’:

    %->%, %<-%


The following objects are masked from ‘package:rtracklayer’:

    blocks, path


The following object is masked from ‘package:Biostrings’:

    union


The following object is masked from ‘package:XVector’:

    path


The following object is masked from ‘package:GenomicRanges’:

    union


The following object is masked from ‘package:IRanges’:

    union


The following object is masked from ‘package:S4Vectors’:

    union


The following objects are masked from ‘package:BiocGenerics’:

    normalize, path, union


The following objects are masked from ‘package:dplyr’:

    as_data_frame, groups, union


The following objects are masked from ‘package:purrr’:

    compose, simplify


The following object is masked from ‘package:tidyr’:

    crossing


The following object is masked from ‘package:tibble’:

    as_data_frame


The following objects a

[1] "2023-07-01 14:16:07 CDT"

In [26]:
print(Sys.time())
    
print("centrality_rankings - Betweenness:")
print(head(centrality_rankings$Betweenness,20))
print("centrality_rankings - PageRank:")
print(head(centrality_rankings$PageRank,20))

print(Sys.time())
print("calculating rank_tfs...")
tf_ranks = rank_tfs(tf_results, 
                    negative_genes = rownames(de_genes_BK)[which(de_genes_BK$logfoldchanges < 0)], 
                    seurat = mpt, 
                    num_cores =  1
    )
print(Sys.time())

[1] "2023-07-01 14:16:07 CDT"
[1] "centrality_rankings - Betweenness:"
       FOXP2        ESRRG        ESRRB        DNMT1        MECOM         SIM2 
27554.535931 17800.117749 11933.733225 11860.931169  6931.570455  5173.581169 
       NPAS2        NR4A1        MEF2C        RUNX2        KLF11         SPI1 
 2562.890476  1016.942857   419.009524   274.000000   212.400000    80.625541 
        ETV5         ESR2         RXRA        KLF15       PKNOX2       POU6F2 
   14.978571    10.666667    10.250000     8.000000     5.533333     3.166667 
         YY1         MBD2 
    3.000000     2.666667 
[1] "centrality_rankings - PageRank:"
      FOXP2       ESRRB       ESRRG       DNMT1      ARNTL2       RUNX1 
0.038507726 0.019048896 0.014741394 0.013761763 0.011713360 0.011352465 
      PPARA       KLF12      BCL11A       MEF2A       TCF12        PAX8 
0.011045549 0.010349724 0.009558086 0.007480452 0.006997164 0.006328284 
       SIM2      SREBF2        PAX2      TFAP2D       BACH2       GLIS3

In [27]:
head(tf_ranks,20)
tail(tf_ranks,20)

,TF_name,Score,SE
,<chr>,<dbl>,<dbl>
ESRRG,ESRRG,15.6534465,0.24697289
FOXP2,FOXP2,6.0024381,0.08152818
MECOM,MECOM,5.8507643,0.21804965
ESRRB,ESRRB,5.1961054,0.08999914
KLF12,KLF12,2.9818846,0.09336354
PPARA,PPARA,2.2986633,0.05867425
PRDM16,PRDM16,2.2879389,0.07655572
RUNX1,RUNX1,2.2287677,0.06390962
MEF2A,MEF2A,1.8178211,0.06985107


,TF_name,Score,SE
,<chr>,<dbl>,<dbl>
ARID3A,ARID3A,-0.1648217,0.01213434
RARB,RARB,-0.1658627,0.02149325
EBF1,EBF1,-0.1661661,0.01384698
VDR,VDR,-0.2023878,0.01119827
TFCP2L1,TFCP2L1,-0.2156912,0.04752411
CUX1,CUX1,-0.2199603,0.02283824
ARID5B,ARID5B,-0.2767234,0.02592344
ESR1,ESR1,-0.3008193,0.01888258
RORA,RORA,-0.3174730,0.05859488


In [28]:
save(list=c("gene_list", "aen_lists", "de_genes_BK", "tf_results", "tf_ranks", "regulator_tf_names", "centrality_rankings", "peak_results"), 
     file = file.path(processed_dir, paste0("RENIN_324701_cells_", FindMarkers_ident_1, "_vs_", FindMarkers_ident_2, ".RData")), compress = T)

In [29]:
Sys.time()

[1] "2023-07-01 14:19:31 CDT"